In [1]:
import torch.nn as nn
from torch.functional import F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)


In [2]:
INPUT_DIM =  20002
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 6
DROPOUT = 0.5
PAD_IDX =  1

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES ,OUTPUT_DIM, DROPOUT, PAD_IDX)

In [3]:
import torch
model.load_state_dict(torch.load('tut2-model.pt'))

#test_loss, test_acc = evaluate(model, test_iterator, criterion)

<All keys matched successfully>

In [4]:
import torchtext.vocab as vocab
from torchtext import data

custom_embeddings = vocab.Vectors(name = 'custom_embeddings/embeddings_conv.txt',
                                  cache = 'custom_embeddings')

 

In [5]:
emdeding_dict = custom_embeddings.stoi

import pickle

TEXT = pickle.load(open('./custom_embeddings/train_data_field', 'rb'))

In [6]:
import spacy
nlp = spacy.load('en')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]

            
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)

    prediction = torch.sigmoid(model(tensor))
    return list(prediction.squeeze().detach())

In [7]:
predict_sentiment(model, "retaxrded quiet or kill you fucking stupid looser")

[tensor(1.0000),
 tensor(0.5322),
 tensor(0.9962),
 tensor(0.6778),
 tensor(0.9962),
 tensor(0.3418)]

In [156]:
dic= {1:'a' ,2 :'b'}
dic.items()

dict_items([(1, 'a'), (2, 'b')])